In [3]:
import sqlite3
import pandas as pd
import glob

In [9]:
# import data

data_dir = '/home/travis/data/s10_database'

data_list = glob.glob(os.path.join(data_dir, '*'))
data_list


scanner3d_data = pd.read_csv('/home/travis/data/s10_database/2020-01-21_3d_volumes_entropy_vs009_fullres_newo3d.csv')
rgb_data = pd.read_csv('/home/travis/data/s10_database/stereoTop_full_season_clustering.csv')
flir_data = pd.read_csv('/home/travis/data/s10_database/s10_flir_rgb_clustering_v4.csv')

In [6]:
# concatonating psii data
all_psii = glob.glob(os.path.join( '/home/travis/data/s10_database/fluorescence_outs', '*', '*'))

['/home/travis/data/s10_database/fluorescence_outs/2020-02-13_fluorescence_aggregation/2020-02-13_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-27_fluorescence_aggregation/2020-02-27_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-07_fluorescence_aggregation/2020-02-07_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-19_fluorescence_aggregation/2020-02-19_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-01-24_fluorescence_aggregation/2020-01-24_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-20_fluorescence_aggregation/2020-02-20_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-12_fluorescence_aggregation/2020-02-12_fluorescence_aggregation.csv',
 '/home/travis/data/s10_database/fluorescence_outs/2020-02-14_fluorescence_aggregation/2020-02-14

In [10]:
psii_data = pd.DataFrame()


for i in all_psii:
    df = pd.read_csv(i)
    psii_data = psii_data.append(df, ignore_index = True)



In [18]:
# get all columns for keys

print('psii:', psii_data.columns)
print('rgb:', rgb_data.columns)
print('flir:', flir_data.columns)
print('3d:', scanner3d_data.columns)

psii: Index(['Unnamed: 0', 'Plot', 'F0', 'FM', 'FV', 'FV/FM'], dtype='object')
rgb: Index(['Unnamed: 0', 'plant_name', 'date', 'treatment', 'plot', 'genotype',
       'lon', 'lat', 'min_x', 'max_x', 'min_y', 'max_y', 'nw_lat', 'nw_lon',
       'se_lat', 'se_lon', 'bounding_area_m2', 'double_lettuce',
       'Unnamed: 0.1', 'id', 'geometry', 'index_right', 'ID'],
      dtype='object')
flir: Index(['index', 'date', 'treatment', 'plot', 'genotype', 'lon', 'lat', 'min_x',
       'max_x', 'min_y', 'max_y', 'nw_lat', 'nw_lon', 'se_lat', 'se_lon',
       'bounding_area_m2', 'roi_temp', 'quartile_1', 'mean', 'median',
       'quartile_3', 'variance', 'std_dev', 'labels', 'plant_name',
       'double_lettuce'],
      dtype='object')
3d: Index(['plant_name', 'min_x', 'min_y', 'min_z', 'max_x', 'max_y', 'max_z',
       'num_points', 'hull_volume', 'oriented_bounding_box',
       'axis_aligned_bounding_box', 'persistence_entropy_0',
       'persistence_entropy_1', 'persistence_entropy_2', 'number_

In [23]:
# rgb needs cleaning

rgb_data = rgb_data.drop('id', 1)

In [26]:
# add rgb to database

with sqlite3.connect('po.db') as con:
    rgb_data.to_sql('rgb', con=con, dtype={('plant_name', 'date'): 'STRING PRIMARY KEY'})

/home/travis/Downloads/yes/envs/general/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [27]:
# add flir

with sqlite3.connect('po.db') as con:
    flir_data.to_sql('flir', con=con, dtype={('plant_name', 'date'): 'STRING PRIMARY KEY'})

In [32]:
# add 3d

with sqlite3.connect('po.db') as con:
    scanner3d_data.to_sql('scanner3d', con=con, dtype={('plant_name', 'date'): 'STRING PRIMARY KEY'})

/home/travis/Downloads/yes/envs/general/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [42]:
con = sqlite3.connect('po.db')

q_result = pd.read_sql_query('select * from scanner3d where plant_name="Aido_100"', con)

q_result.head()

,index,plant_name,min_x,min_y,min_z,max_x,max_y,max_z,num_points,hull_volume,...,amplitude_silhouette_1,amplitude_silhouette_2,amplitude_heat_0,amplitude_heat_1,amplitude_heat_2,amplitude_persistence_image_0,amplitude_persistence_image_1,amplitude_persistence_image_2,Unnamed: 0,date
0,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,0.001329,0.000247,266.368258,84.710873,2.026012,997.683102,618.368853,34.529446,None,2020-01-21


In [44]:
rgb_result = pd.read_sql_query('select * from rgb where plant_name="Aido_100"', con)
rgb_result.head()


,index,Unnamed: 0,plant_name,date,treatment,plot,genotype,lon,lat,min_x,...,nw_lat,nw_lon,se_lat,se_lon,bounding_area_m2,double_lettuce,Unnamed: 0.1,geometry,index_right,ID
0,349389,381081,Aido_100,2019-12-21,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974951,33.074785,266.0,...,33.074785,-111.974951,33.074785,-111.97495,0.004169,0.0,None,None,None,None
1,349421,381113,Aido_100,2019-12-30,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974951,33.074785,187.0,...,33.074784,-111.974951,33.074785,-111.97495,0.008833,0.0,None,None,None,None
2,349481,381174,Aido_100,2020-01-02,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974951,33.074785,244.0,...,33.074785,-111.974951,33.074785,-111.97495,0.008755,0.0,None,None,None,None
3,349551,381245,Aido_100,2020-01-05,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974951,33.074785,232.0,...,33.074785,-111.974951,33.074785,-111.97495,0.009831,0.0,None,None,None,None
4,349634,381332,Aido_100,2020-01-07,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974951,33.074785,218.0,...,33.074785,-111.974951,33.074785,-111.97495,0.009996,0.0,None,None,None,None


In [46]:
flir_result = pd.read_sql_query('select * from flir where plant_name="Aido_100"', con)
flir_result.head()

,level_0,index,date,treatment,plot,genotype,lon,lat,min_x,max_x,...,roi_temp,quartile_1,mean,median,quartile_3,variance,std_dev,labels,plant_name,double_lettuce
0,193028,17550,2020-02-03,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974950,33.074783,53,125,...,289.920197,290.060730,290.808044,290.481110,291.349396,1.103919,1.050676,0,Aido_100,0.0
1,193032,12072,2020-02-04,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974949,33.074783,68,145,...,281.195343,280.139221,282.111633,282.428162,283.741058,3.768269,1.941203,0,Aido_100,0.0
2,193035,2469,2020-02-05,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974950,33.074783,62,133,...,284.853821,284.998688,286.550079,286.198303,288.041046,3.409416,1.846460,0,Aido_100,0.0
3,193039,17058,2020-02-08,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974949,33.074783,74,145,...,296.314453,295.517670,297.813507,297.673111,299.624886,6.175176,2.484990,0,Aido_100,0.0
4,193047,12818,2020-02-11,treatment 1,MAC_Field_Scanner_Season_10_Range_8_Column_20,Aido,-111.974949,33.074783,59,135,...,294.674805,292.082489,293.581085,293.364502,294.904266,2.625972,1.620485,0,Aido_100,0.0


In [57]:
# join on 3d since it has the smallest number


union_result = pd.read_sql_query('SELECT * FROM scanner3d LEFT OUTER JOIN rgb on scanner3d.plant_name = rgb.plant_name WHERE scanner3d.plant_name = "Aido_100"', con)
union_result.head()

,index,plant_name,min_x,min_y,min_z,max_x,max_y,max_z,num_points,hull_volume,...,nw_lat,nw_lon,se_lat,se_lon,bounding_area_m2,double_lettuce,Unnamed: 0.1,geometry,index_right,ID
0,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.004169,0.0,None,None,None,None
1,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074784,-111.974951,33.074785,-111.97495,0.008833,0.0,None,None,None,None
2,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.008755,0.0,None,None,None,None
3,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.009831,0.0,None,None,None,None
4,0,Aido_100,-0.646141,-0.695795,-0.175155,0.642084,0.654622,0.432095,54945.0,0.546489,...,33.074785,-111.974951,33.074785,-111.97495,0.009996,0.0,None,None,None,None
